In [1]:
!pip install -U textblob

Requirement already up-to-date: textblob in /home/nbuser/anaconda3_420/lib/python3.5/site-packages
Requirement already up-to-date: nltk>=3.1 in /home/nbuser/anaconda3_420/lib/python3.5/site-packages (from textblob)
Requirement already up-to-date: six in /home/nbuser/anaconda3_420/lib/python3.5/site-packages (from nltk>=3.1->textblob)
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/nbuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nbuser/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [17]:
# Read data
import numpy as np
import pandas as pd

df = pd.read_csv('/home/nbuser/library/narendramodi/data/nm_tweets_stream.csv', na_values=['#NAME?'])

In [18]:
# Look at the data
print(df.head(5))

                       created_at      screen_name  \
0  Tue Apr 03 06:06:00 +0000 2018        PrashnikQ   
1  Tue Apr 03 06:06:00 +0000 2018  BRIJBHAN0304198   
2  Tue Apr 03 06:06:01 +0000 2018          sudar2k   
3  Tue Apr 03 06:06:01 +0000 2018        nmeena970   
4  Tue Apr 03 06:06:02 +0000 2018      JainSreepal   

                                          tweet_text time_zone  
0  @BJP4India @Sanju_Verma_ @narendramodi @Piyush...       NaN  
1  I received Birthday Greetings directly from PM...       NaN  
2  RT @_ugra_: Fear on Chinese maami's face is pa...       NaN  
3  RT @imNitinRajput: @manakgupta @noidapolice @U...       NaN  
4  @dna @rsprasad sir link aadhar with voter I'd ...       NaN  


In [19]:
# create copy of the dataset.
df_copy = df.copy()
df_copy['created_at'] = pd.to_datetime(df['created_at']).dt.strftime("%Y%m%d%H%M%S")




In [20]:
# Find out sentiment from the tweets and assign this to a attribute.
from textblob import TextBlob

df_copy['sentiment'] = [0 if TextBlob(str(tweets)).sentiment.polarity == 0.0 else 1 if TextBlob(str(tweets)).sentiment.polarity > 0.0 else -1 for tweets in df_copy['tweet_text']]
df_copy = df_copy.drop('tweet_text', 1)



In [21]:
# Look at the output of the sentiment 
df_copy.head(10)

,created_at,screen_name,time_zone,sentiment
0,20180403060600,PrashnikQ,NaN,0
1,20180403060600,BRIJBHAN0304198,NaN,1
2,20180403060601,sudar2k,NaN,0
3,20180403060601,nmeena970,NaN,0
4,20180403060602,JainSreepal,NaN,1
5,20180403060603,NSUIHimachal,Chennai,0
6,20180403060603,shivasen20,Pacific Time (US & Canada),0
7,20180403060603,Shah2016Aashish,NaN,0
8,20180403060604,singhshesh9525,NaN,0
9,20180403060606,Rajendr21934396,NaN,0


In [22]:
# Decide which categorical variables you want to use in model
for col_name in df_copy.columns:
    if df_copy[col_name].dtypes == 'object' or df_copy[col_name].dtypes == 'int64' :
        unique_cat = len(df_copy[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))

Feature 'created_at' has 30871 unique categories
Feature 'screen_name' has 37150 unique categories
Feature 'time_zone' has 124 unique categories
Feature 'sentiment' has 3 unique categories


In [33]:
#create new datafrmae for timezone sentument.
df_zone_sentiment = df_copy[['time_zone','sentiment']]

In [24]:
# Function to dummy all the categorical variables used for modeling
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [34]:
todummy_list = ['time_zone','sentiment']
df_zone_sentiment = dummy_df(df_zone_sentiment[todummy_list], todummy_list)


In [35]:
df_zone_sentiment.head(10)

,time_zone_Abu Dhabi,time_zone_Adelaide,time_zone_Africa/Asmara,time_zone_Africa/Khartoum,time_zone_Africa/Nairobi,time_zone_Alaska,time_zone_Almaty,time_zone_America/Chicago,time_zone_America/Los_Angeles,time_zone_America/New_York,...,time_zone_Tokyo,time_zone_UTC,time_zone_Urumqi,time_zone_Warsaw,time_zone_Wellington,time_zone_West Central Africa,time_zone_Yerevan,sentiment_-1,sentiment_0,sentiment_1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
